In [1]:
import pandas as pd
import numpy as np
from pydataset import data
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import math

### Exercises
For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like. Note that some of the questions are intentionally phrased in a vague way. It is your job to reword these as more precise questions that could be tested.

**Has the network latency gone up since we switched internet service providers?**

* **H<sub>0</sub>:** The internet service provider does NOT affect the network latency

* **H<sub>1</sub>:** The internet service provider does affect the network latency

* **True Positive:** The internet service provider does affect the network latency, & we were correct

* **True Negative:** The internet service provider does NOT affect the network latency, & we were correct

* **Type I:** The internet service provider does affect the network latency, & we were WRONG

* **Type II:** The internet service provider does NOT affect the network latency, & we were WRONG

**Is the website redesign any good?**

* **H<sub>0</sub>:**

* **H<sub>1</sub>:**

* **True Positive:**

* **True Negative:**

* **Type I:**

* **Type II:**

**Is our television ad driving more sales?**

* **H<sub>0</sub>:**

* **H<sub>1</sub>:**

* **True Positive:**

* **True Negative:**

* **Type I:**

* **Type II:**

**1. Use the following contingency table to help answer the question of whether using a Macbook and being a Codeup student are independent of each other.**

- $H_o$: There is NO relationship between using a Macbook & being a Codeup student
- $H_a$: There IS a relationship between using a Macbook & being a Codeup student

In [254]:
data_dict = {'Codeup Student': [49,1], 'Not Codeup Student': [20,30]}
data_dict

{'Codeup Student': [49, 1], 'Not Codeup Student': [20, 30]}

In [257]:
observed = pd.DataFrame(data_dict, index=['Macbook','Doesn\'t Use A Macbook'])
observed

,Codeup Student,Not Codeup Student
Macbook,49,20
Doesn't Use A Macbook,1,30


In [258]:
α = .05

In [261]:
chi2, p, dof, expected = stats.chi2_contingency(observed)

In [268]:
print('Observed')
print(observed.values)

print('\nExpected')
print(expected)

print(f'\nchi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

Observed
[[49 20]
 [ 1 30]]

Expected
[[34.5 34.5]
 [15.5 15.5]]

----
chi^2 = 36.6526
p     = 0.0000


In [266]:
if p < α:
    print('reject the null hypothesis')
else:
    print('fail to reject the null hypothesis')

reject the null hypothesis


> We have concluded that there IS a relationship between using a Macbook & being a Codeup student

**2. Choose another 2 categorical variables from the mpg dataset.**

In [270]:
df = data('mpg')
df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [275]:
df = df.rename(columns = {'class':'class_type'})

In [276]:
observed = pd.crosstab(df.cyl,df.class_type)
observed

class_type,2seater,compact,midsize,minivan,pickup,subcompact,suv
cyl,,,,,,,
4,0,32,16,1,3,21,8
5,0,2,0,0,0,2,0
6,0,13,23,10,10,7,16
8,5,0,2,0,20,5,38


* State your null and alternative hypotheses.

> $H_o$: There is NO relationship between the amount of cylinders & class_type

>$H_a$: There IS a relationship between the amount of cylinders & class_type

* State your alpha.

In [277]:
α = .05

* Perform a $\chi^2$ test of independent.

In [281]:
chi2, p, dof, expected = stats.chi2_contingency(observed)

1.5351076620141522e-20

In [282]:
if p < α:
    print('reject the null hypothesis')
else:
    print('fail to reject the null hypothesis')

reject the null hypothesis


* State your conclusion

> We have concluded that there IS a relationship between the amount of cylinders & class_type.

Use the data from the employees database to answer these questions:

* Is an employee's gender independent of whether an employee works in sales or marketing? (only look at current employees)

>$H_o$: There is NO relationship between an employee's gender & whether they work in sales or marketing

>$H_a$: There IS a relationship between an employee's gender & whether they work in sales or marketing

In [299]:
import env
from env import get_db_url

In [300]:
url = env.get_db_url(env.username,env.host,env.password,'employees')

In [295]:
query = '''
SELECT dept_name,gender
FROM departments d
JOIN dept_emp de ON d.dept_no = de.dept_no
JOIN employees e ON e.emp_no = de.emp_no
WHERE de.to_date > NOW()
  AND d.dept_name IN ('Customer Service', 'Sales');
'''
df = pd.read_sql(query, url)

In [90]:
α = .05

In [298]:
observed = pd.crosstab(df.gender,df.dept_name)
observed

dept_name,Customer Service,Sales
gender,,
F,7007,14999
M,10562,22702


In [96]:
chi2, p, dof, expected = stats.chi2_contingency(observed)

In [97]:
if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

failed to reject the null hypothesis


> We have concluded that there is NO relationship between an employee's gender & whether they work in sales or marketing

* Is an employee's gender independent of whether or not they are or have been a manager?

In [323]:
query = '''
select *
from dept_manager dm
right join employees e
	ON dm.emp_no = e.emp_no
order by dm.emp_no DESC;
'''
df = pd.read_sql(query, url)

In [324]:
df 

,emp_no,dept_no,from_date,to_date,emp_no,birth_date,first_name,last_name,gender,hire_date
0,111939.0,d009,1996-01-03,9999-01-01,111939,1960-03-25,Yuchang,Weedman,M,1989-07-10
1,111877.0,d009,1992-09-08,1996-01-03,111877,1962-10-18,Xiaobin,Spinelli,F,1991-08-17
2,111784.0,d009,1988-10-17,1992-09-08,111784,1956-06-14,Marjo,Giarratana,F,1988-02-12
3,111692.0,d009,1985-01-01,1988-10-17,111692,1954-10-05,Tonny,Butterworth,F,1985-01-01
4,111534.0,d008,1991-04-08,9999-01-01,111534,1952-06-27,Hilary,Kambil,F,1988-01-31
...,...,...,...,...,...,...,...,...,...,...
300019,NaN,None,None,None,492778,1962-09-21,Kasturi,Lakshmanan,M,1986-12-01
300020,NaN,None,None,None,492779,1954-07-29,Wilmer,Matteis,M,1998-03-17
300021,NaN,None,None,None,492780,1955-06-05,Mana,Erva,F,1992-06-23
300022,NaN,None,None,None,492781,1955-01-12,Khedija,Fritzsche,M,1989-11-12


In [326]:
df['is_manager'] = df.to_date.notnull()
df

,emp_no,dept_no,from_date,to_date,emp_no,birth_date,first_name,last_name,gender,hire_date,is_manager
0,111939.0,d009,1996-01-03,9999-01-01,111939,1960-03-25,Yuchang,Weedman,M,1989-07-10,True
1,111877.0,d009,1992-09-08,1996-01-03,111877,1962-10-18,Xiaobin,Spinelli,F,1991-08-17,True
2,111784.0,d009,1988-10-17,1992-09-08,111784,1956-06-14,Marjo,Giarratana,F,1988-02-12,True
3,111692.0,d009,1985-01-01,1988-10-17,111692,1954-10-05,Tonny,Butterworth,F,1985-01-01,True
4,111534.0,d008,1991-04-08,9999-01-01,111534,1952-06-27,Hilary,Kambil,F,1988-01-31,True
...,...,...,...,...,...,...,...,...,...,...,...
300019,NaN,None,None,None,492778,1962-09-21,Kasturi,Lakshmanan,M,1986-12-01,False
300020,NaN,None,None,None,492779,1954-07-29,Wilmer,Matteis,M,1998-03-17,False
300021,NaN,None,None,None,492780,1955-06-05,Mana,Erva,F,1992-06-23,False
300022,NaN,None,None,None,492781,1955-01-12,Khedija,Fritzsche,M,1989-11-12,False


In [327]:
observed = pd.crosstab(df.is_manager,df.gender)
α = .05

In [328]:
chi2, p, dof, expected = stats.chi2_contingency(observed)

In [329]:
print('Observed')
print(observed.values)
print('\nExpected')
print(expected.astype(int))
print('\n----')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

Observed
[[120038 179962]
 [    13     11]]

Expected
[[120041 179958]
 [     9     14]]

----
chi^2 = 1.4567
p     = 0.2275


In [330]:
if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

failed to reject the null hypothesis


>We have concluded that there is NO relationship between an employee's gender & if they are/have been a manager

# Correlation

### 1. Answer with the type of stats test you would use (assume normal distribution):

* Is there a relationship between the length of your arm and the length of your foot?

> Pearson's R

* Does smoking affect when or not someone has lung cancer?

> $\chi^2$

* Is gender independent of a person’s blood type?

> $\chi^2$

* Does whether or not a person has a cat or dog affect whether they live in an apartment?

> $\chi^2$

* Does the length of time of the lecture correlate with a student's grade?

> Pearson's R

### 2. Use the `telco_churn` data.

In [8]:
import env
url = env.get_db_url('telco_churn')

In [164]:
query = '''
select *
from customers
join internet_service_types
	using (internet_service_type_id);
'''
df = pd.read_sql(query, url)

* Does tenure correlate with monthly charges?

In [165]:
r, p = stats.pearsonr(df.tenure,df.monthly_charges)
α = .05

In [166]:
if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


> There is a linear relationship between tenure & monthly charges

* Total charges?

In [167]:
df['total_charges'] = df.total_charges.str.strip()

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   internet_service_type_id  7043 non-null   int64  
 1   customer_id               7043 non-null   object 
 2   gender                    7043 non-null   object 
 3   senior_citizen            7043 non-null   int64  
 4   partner                   7043 non-null   object 
 5   dependents                7043 non-null   object 
 6   tenure                    7043 non-null   int64  
 7   phone_service             7043 non-null   object 
 8   multiple_lines            7043 non-null   object 
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

In [169]:
df = df[df.total_charges != '']

In [170]:
df['total_charges'] = df.total_charges.astype(float)

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   internet_service_type_id  7032 non-null   int64  
 1   customer_id               7032 non-null   object 
 2   gender                    7032 non-null   object 
 3   senior_citizen            7032 non-null   int64  
 4   partner                   7032 non-null   object 
 5   dependents                7032 non-null   object 
 6   tenure                    7032 non-null   int64  
 7   phone_service             7032 non-null   object 
 8   multiple_lines            7032 non-null   object 
 9   online_security           7032 non-null   object 
 10  online_backup             7032 non-null   object 
 11  device_protection         7032 non-null   object 
 12  tech_support              7032 non-null   object 
 13  streaming_tv              7032 non-null   object 
 14  streaming_mov

In [172]:
stats.pearsonr(df.tenure,df.total_charges)
α = .05

In [173]:
if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


> There is a linear relationship between tenure & total charges

* What happens if you control for phone and internet service?

In [184]:
df['has_internet'] = df.internet_service_type != 'None'

In [185]:
df.has_internet.value_counts()

has_internet
True     5512
False    1520
Name: count, dtype: int64

In [186]:
df[['phone_service','tenure','monthly_charges','total_charges','has_internet']]

,phone_service,tenure,monthly_charges,total_charges,has_internet
0,Yes,9,65.60,593.30,True
1,Yes,9,59.90,542.40,True
2,Yes,4,73.90,280.85,True
3,Yes,13,98.00,1237.85,True
4,Yes,3,83.90,267.40,True
...,...,...,...,...,...
7038,Yes,13,55.15,742.90,True
7039,Yes,22,85.10,1873.70,True
7040,Yes,2,50.30,92.75,True
7041,Yes,67,67.85,4627.65,True


In [195]:
with_internet = df[df.has_internet == True]
without_internet = df[df.has_internet == False]

with_phone_service = df[df.phone_service == 'Yes']
without_phone_service = df[df.phone_service == 'No']

In [198]:
stats.pearsonr(with_phone_service.tenure,with_phone_service.monthly_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [201]:
stats.pearsonr(without_phone_service.tenure,without_phone_service.monthly_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [202]:
stats.pearsonr(with_phone_service.tenure,with_phone_service.total_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [203]:
stats.pearsonr(without_phone_service.tenure,without_phone_service.total_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [207]:
stats.pearsonr(with_internet.tenure,with_internet.monthly_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [206]:
stats.pearsonr(with_internet.tenure,with_internet.total_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [204]:
stats.pearsonr(without_internet.tenure,without_internet.monthly_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


In [205]:
stats.pearsonr(without_internet.tenure,without_internet.total_charges)
α = .05

if p < α:
    print('reject the null hypothesis')
else:
    print('failed to reject the null hypothesis')

reject the null hypothesis


### 3. Use the employees database.

In [210]:
url = env.get_db_url('employees')

In [211]:
query = '''
select *
 from employees
join salaries
using(emp_no);
'''
df = pd.read_sql(query, url)

* Is there a relationship between how long an employee has been with the company and their salary?

In [212]:
df

,emp_no,birth_date,first_name,last_name,gender,hire_date,salary,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,60117,1986-06-26,1987-06-26
1,10001,1953-09-02,Georgi,Facello,M,1986-06-26,62102,1987-06-26,1988-06-25
2,10001,1953-09-02,Georgi,Facello,M,1986-06-26,66074,1988-06-25,1989-06-25
3,10001,1953-09-02,Georgi,Facello,M,1986-06-26,66596,1989-06-25,1990-06-25
4,10001,1953-09-02,Georgi,Facello,M,1986-06-26,66961,1990-06-25,1991-06-25
...,...,...,...,...,...,...,...,...,...
2844042,499999,1958-05-01,Sachin,Tsukuda,M,1997-11-30,63707,1997-11-30,1998-11-30
2844043,499999,1958-05-01,Sachin,Tsukuda,M,1997-11-30,67043,1998-11-30,1999-11-30
2844044,499999,1958-05-01,Sachin,Tsukuda,M,1997-11-30,70745,1999-11-30,2000-11-29
2844045,499999,1958-05-01,Sachin,Tsukuda,M,1997-11-30,74327,2000-11-29,2001-11-29


* Is there a relationship between how long an employee has been with the company and the number of titles they have had?